In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("QR decomposition.ipynb")

# Matrix Analysis 2024 - EE312

## Week 12 - QR DECOMPOSITION
[LTS2](https://lts2.epfl.ch)

### Objectives
The goal of this week's exercise session is to study some aspects of the QR decomposition and its connections to projections and inverse. We will study two methods that can be used to compute the QR decomposition of a matrix and an application to eigendecomposition.

Please submit your answers **individually**.

Let us consider a square $n\times n$ real matrix $A$ having linearly independent columns (NB: QR factorization is applicable to rectangular matrices, we consider square matrices for simplification). The QR decomposition of $A$ is $A=QR$ where $Q$ is an orthogonal matrix and $R$ and upper-triangular matrix.



# 1. Gram-Schmidt orthogonalization

Reminder:
- The projection operator of a vector $v$ on another $u$ is given by $P_u v = \frac{\langle u,v \rangle}{\langle u,u \rangle}u$.
- Gram-Schmidt orthogonalization process of a basis $(v_1, v_2, ..., v_{n})$ produces an orthormal basis $(e_1, e_2, ..., e_{n})$ as follows:

$u_1 = v_1$, $e_1 = \frac{u_1}{\|u_1\|}$

$u_2 = v_2 - P_{u_1}v_2$, $e_2 = \frac{u_2}{\|u_2\|}$

$u_3 = v_3 - P_{u_2}v_3 - P_{u_1}v_3$, $e_3 = \frac{u_3}{\|u_3\|}$

...

$u_k = v_k - \sum_{j=1}^{k-1} P_{u_j}v_{k}$, $e_k = \frac{u_k}{\|u_k\|}$

<!-- BEGIN QUESTION -->

## 1.1 
We perform the QR decomposition of $A$ by applying Gram-Schmidt to its column vectors of $A$ (denoted by $a_k, k=1,...n)$. $Q$ is made of the column vectors $e_k$, i.e. $Q=(e_1|e_2|...|e_{n})$. Let us denote by $r_{jk}$ the coefficients of $R$.

Prove that $r_{kk} = \|u_k\|$, $r_{jk} = <e_j, a_k>$ for $j<k$ and $r_{jk} = 0$ for $j>k$.

_Type your answer here, replacing this text._

Using the Gram-Schmidt formula, we can write:
$a_k = u_k + \sum_{j=1}^{k-1} P_{u_j}a_{k} = u_k + \sum_{j=1}^{k-1}\frac{<u_j, a_k>}{\|u_j\|^2}u_j = \|u_k\|e_k + \sum_{j=1}^{k-1}\frac{<u_j, a_k>}{\|u_j\|^2}\|u_j\|e_j = \|u_k\|e_k + \sum_{j=1}^{k-1}\frac{<u_j, a_k>}{\|u_j\|}e_j = \|u_k\|e_k + \sum_{j=1}^{k-1}<e_j, a_k>e_j $

Using the definition of $Q$ and $R$ we also have $a_k = \sum_{j=1}^n r_{jk} e_j$. Using the equality above, this proves that $r_{kk} = \|u_k\|$, $r_{jk} = <e_j, a_k>$ for $j<k$ and $r_{jk} = 0$ for $j>k$.


<!-- END QUESTION -->

## 1.2
Implement a function that performs the QR decomposition of a square $n\times n$ matrix using the Gram-Schmidt orthogonalization process

In [50]:
import numpy as np

In [51]:
def qr_decomp_gs(A):
    """
    Performs the QR decomposition of the matrix using Gram-Schmidt
    
    Parameters: 
      - A: input matrix nxn
      
    Returns:
      - Q matrix (orthogonal)
      - R matrix (upper triangular)
    """

    n = A.shape[0]
    Q = np.zeros((n,n))
    R = np.zeros((n,n))
    # BEGIN SOLUTION
    U = np.zeros((n,n))
    U[:, 0] = A[:, 0]
    Q[:, 0] = A[:, 0]/np.linalg.norm(A[:, 0])
    R[0, 0] = np.linalg.norm(A[:, 0])
    for k in np.arange(1, n):
        acc = np.zeros((n))
        for j in np.arange(0, k):
            R[j, k] = np.dot(A[:, k], Q[:, j])
            acc = acc + (np.dot(A[:, k], U[:, j])/np.dot(U[:, j], U[:, j]))*U[:, j]

        U[:, k] = A[:, k] - acc
        Q[:, k] = U[:, k]/np.linalg.norm(U[:, k])
        R[k, k] = np.linalg.norm(U[:, k])
    return Q,R
    # END SOLUTION

In [ ]:
grader.check("q1p2")

## 1.3
Using the properties of Q, write a routine that can solve a linear system $Ax=b$ using the QR factorization of $A$ ($A$ being a $n\times n$ real matrix).
What property do the coefficients of $R$ need to satisfy for the solution to exist ?

We have $Ax = b$ and $A=QR$ with $Q$ orthogonal hence $Q^TQRx = Q^Tb$ i.e. $Rx=Q^Tb$. 

Given $R$ is upper triangular, we do not need to perform compute $R^{-1}$ explicitely and the system can be solved using backsubstitution. 
Since we divide by $r_{kk}$ to get the $x_k$ values, those diagonal coefficients have to be non-zero.

In [57]:
def solve_gs(A, b):
    """
    Solve a linear system Ax=b using QR decomposition
    
    Parameters:
      - A input matrix nxn
      - b result vector (length n)
    
    Returns:
      - solution vector x (length n)
    """
    n = A.shape[0]
    x = np.zeros(n)
    # BEGIN SOLUTION
    Q, R = qr_decomp_gs(A)
    c = Q.T@b
    x = np.zeros(n)
    x[n-1] = c[n-1]/R[n-1, n-1]
    for k in np.arange(n-2, -1, -1):
        s = np.dot(R[k, k+1:], x[k+1:])
        x[k] = (c[k] - s)/R[k,k]
    return x
    # END SOLUTION

In [58]:
# Small numerical example
A=np.array([[12,-51,4],[6,167,-68],[-4,24,-41]])
b=np.array([1, -1, 1])

In [60]:
x = solve_gs(A, b)

In [ ]:
A@x

In [ ]:
grader.check("q1p3")

<!-- BEGIN QUESTION -->

## 1.4
Consider the matrix 
$B=\begin{pmatrix}1 & 1 & 1 \\ \varepsilon & 0 & 0\\ 0 & \varepsilon & 0\end{pmatrix}$, with $\varepsilon$ being small enough (typically $10^{-8}$ or smaller). In that case, is the method used to compute Q and R still reliable (and why) ?

In [ ]:
eps=1e-8

_Type your answer here, replacing this text._

In [ ]:
B = np.array([[1,1,1],[eps,0,0],[0,eps,0]])# SOLUTION

In [ ]:
# Check the orthogonality of Q
QB@QB.T # SOLUTION

In [ ]:
# Check the validity of the decomposition
QB@RB-B # SOLUTION

We still have $QR=B$, but $Q$ is no longer orthogonal ! This is due to the numerical precision of the computer, that rounds off the norm of the first column of $B$ to 1, when it should be $\sqrt{1+\varepsilon^2}$.

<!-- END QUESTION -->

# 2. Householder reflections

We will now study and implement another method to perform the QR decomposition that is more resistant to the numerical issues mentioned above.

<!-- BEGIN QUESTION -->

## 2.1 
Let $v$ be a unit vector and the associated transform $H_v=I-2vv^T$. Prove that $H_v$ is orthogonal and that it preserves the norm. What is the effect of $H_v$ on a vector $u$ orthogonal to $v$ ?

_Type your answer here, replacing this text._

$H_vH_v^T = (I - 2vv^T)(I - 2vv^T)^T = (I - 2vv^T)(I - 2vv^T) = I -4vv^T + 4v(v^Tv)v^T$

Since $v$ is a unit vector, $v^Tv=1$ i.e. $H_vH_v^T = I - 4vv^T + 4vv^T = I$.

$\|H_vu\|^2 = (H_vu)^TH_vu = u^TH_v^TH_vu =  u^Tu = \|u\|^2$, $H_v$, as any orthogonal transform preserves norm. 

If $u$ is orthogonal to $v$, we have $v^Tu=0$.
$H_vu = u - 2 vv^Tu = u$. $H_v$ does not modify vectors orthogonal to $v$.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## 2.2 
What is the effect of $H_v$ on any vector $u$ (drawing the planar case can be of help).

_Type your answer here, replacing this text._

$H_vu = u - 2vv^Tu = u - 2<v,u>v$

The term $<v,u>v$ is in fact the orthogonal projection of $u$ on $v$ (remember $<v,v>=\|v\|=1)$. If we draw the effect of $H_v$ on $u$, it yields the reflection of $u$ around $v^\perp$.

![householder](images/householder.png)

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## 2.3
The goal is now to design $n$ reflection operators $H_{v_1}, H_{v_2}, ..., H_{v_n}$ s.t. we have
$$
H_{v_n}...H_{v_2}H_{v_1}A = R,
$$
where $R$ is an upper triangular matrix. 

The reflection operations are meant to be "progressive", i.e. $H_{v_k}...H_{v_1}A$ has its $k$ first columns upper triangular.

- Find $v_1$ and $\alpha$ s.t. $H_{v_1}a_1 = \alpha e_1$, where $a_1$ is the first column of $A$ and $e_1 = \begin{pmatrix}1\\0\\ \vdots \\0 \end{pmatrix}$.

- Let us now generalize the previous step to find $H_{v_k}$. Given a vector $x = \begin{pmatrix}x^U \\ x^L\end{pmatrix}$ with $x^U \in\mathbb{R}^k$
and $x^L\in\mathbb{R}^{n-k}$. 

Find a vector $v_k$ s.t. $H_{v_k}x = \begin{pmatrix}x^U \\ \alpha e_1^L \end{pmatrix}$, where $\alpha\in\mathbb{R}$ and 
$e_1^L = \begin{pmatrix}1\\0\\ \vdots \\0 \end{pmatrix} \in \mathbb{R}^{n-k}$. 

It might be of help to write $v_k=\begin{pmatrix}v_k^U\\v_k^L\end{pmatrix}$, with $v_k^U \in\mathbb{R}^k$
and $v_k^L\in\mathbb{R}^{n-k}$. 

_Type your answer here, replacing this text._

- $H_{v_1}a_1 = (I -2v_1v_1^T)a_1 = \alpha e_1$, i.e. $a_1 - 2v_1v_1^Ta_1 = \alpha e_1$, 
and finally $a_1 - \alpha e_1 = 2 <v_1, a_1> v_1$, which implies that $v_1$ is colinear to $a_1 - \alpha e_1$.
Remembering that $H_{v_1}$ preserves norm, we have $\|H_{v_1}a_1\| = \|a_1\| = |\alpha|$.
Therefore $v_1 = \frac{1}{\|a_1 \pm \|a_1\|e_1\|}(a_1 \pm \|a_1\|e_1)$ 

-  Since we have $H_{v_k} = I - 2v_kv_k^T = I - 2\begin{pmatrix}v_k^U\\v_k^L\end{pmatrix}\begin{pmatrix}(v_k^U)^T & (v_k^L)^T\end{pmatrix}$, which leads to $H_{v_k} = I - 2\begin{pmatrix}v_k^U(v_k^U)^T & v_k^U(v_k^L)^T \\ (v_k^U)^Tv_k^L & v_k^L(v_k^L)^T\end{pmatrix}$

$H_{v_k}x = \begin{pmatrix}x^U \\ x^L\end{pmatrix} - 2\begin{pmatrix}v_k^U(v_k^U)^T & v_k^U(v_k^L)^T \\ (v_k^U)^Tv_k^L & v_k^L(v_k^L)^T\end{pmatrix}\begin{pmatrix}x^U \\ x^L\end{pmatrix}$

If we want the upper part of $x$ to be preserved, it is relatively obvious that $v_k^U = 0$ will be needed, which leads to
$H_{v_k}x = \begin{pmatrix}x^U \\ x^L\end{pmatrix} - 2\begin{pmatrix}0 & 0 \\ 0 & v_k^L(v_k^L)^T\end{pmatrix}\begin{pmatrix}x^U \\ x^L\end{pmatrix}$.

The lower part yields: $x^L - 2 v_k^L(v_k^L)^Tx^L = x^L - 2<v_k^L, x^L>v_k^L  = \alpha e_1^L$. As for $v_1$, we see that $v_k$ is colinear to $x^L - \alpha e_1^L$, and the norm preservation property yields $\alpha = \pm\|x^L\|$.

Finally, $v_k^L = \frac{1}{\|x^L \pm \|x^L\|e_1^L \|}(x^L \pm \|x^L\|e_1^L)$

<!-- END QUESTION -->

## 2.4
We now have all we need to compute the $H_{v_k}$ matrices. Implement the QR decomposition using the above results. 
You should have found that the sign of $\alpha$ can be either positive or negative. Choose $\alpha$ to have the sign opposite to the one of $x^L_k[0]$. (Use the `numpy.sign`function).

Try your implementation on the $B$ matrix seen in ex. 1. What is the advantage of this implementation ?

In [ ]:
def qr_decomp_hh(A):
    """
    Performs the QR decomposition of the matrix using Householder reflections
    
    Parameters: 
      - A: input matrix nxn
      
    Returns:
      - Q matrix (orthogonal)
      - R matrix (upper triangular)
    """
    n = A.shape[0]
    # BEGIN SOLUTION
    E = np.eye(n)
    Q = np.eye(n)
    R = A.copy()
    for k in np.arange(0, n):
        u = np.zeros(n)
        s = np.sign(R[k,k])
        u[k:] = R[k:,k] + s*np.linalg.norm(R[k:,k])*E[k:, k]
        v = u/np.linalg.norm(u)
        H = E - 2*np.outer(v, v)
        R = H@R
        Q = H@Q
    return Q.T,R # in this case we need the 'inverse' of Q
    # END SOLUTION

In [ ]:
qh, rh = qr_decomp_hh(B)

In [ ]:
# Check orthogonality of Q
qh@qh.T # SOLUTION

In [ ]:
# Check validity of solution
qh@rh-B # SOLUTION

The QR factorization using the Householder reflections is now giving correct results on $B$. This seems a better choice in terms of numerical stability. 

# 3. Eigendecomposition and QR

We will now study some aspects of how to actually perform (numerically) the eigendecomposition of a matrix. 

In the following we will suppose that $A$ has distinct eigenvalues, and we will write them as $\lambda_1, ..., \lambda_n$ s.t. $|\lambda_1| > |\lambda_2|...>|\lambda_n|$, and denote their associated normalized eigenvectors as $v_1, v_2..., v_n$.

<!-- BEGIN QUESTION -->

## 3.1 Power method

Let us consider a vector $w\in\mathbb{R}^n$ of norm 1, and its associated eigendecomposition $w = \sum_{k=1}^n\alpha_k v_k$. 

Compute $A^pw$ (for $p\in\mathbb{N}$, $k>0$). Assuming that $\alpha_1\neq 0$ for the chosen $w$, use this result to find an algorithm to compute $\lambda_1$ and $v_1$ (you may want to introduce another vector $x\in\mathbb{R}^n$ s.t. $x^Tv_1\neq 0$ and consider the quantity $x^TA^pw$) ?

How would you proceed for $\lambda_2$ and $v_2$ and the other eigenvalues ?

What are the limitations of this method ?

_Type your answer here, replacing this text._

Since we have $Av_k = \lambda_kv_k$, we have $Aw = \sum_{k=1}^n\alpha_k\lambda_kv_k$. If we repeat the operation, we easily get $A^pw = \sum_{k=1}^n\alpha_k\lambda_k^pv_k$, which in turn can be rewritten as 
$$
A^pw = \lambda_1^p \left[\alpha_1v_1 + \sum_{k=2}^n\alpha_k\left(\frac{\lambda_k}{\lambda_1}\right)^pv_k\right]
$$

Since $\left|\frac{\lambda_k}{\lambda_1}\right|<1$ for $k>1$, when $p\to+\infty$ those terms will vanish. While this is interesting, this would not be directly applicable to estimate $\lambda_1$, since the power of $\lambda_1$ itself can grow (or vanish) quickly.

If we consider the scalar $x^TA^pw$, we have
$$
x^TA^pw = \lambda_1^p\left[\alpha_1x^Tv_1 + \sum_{k=2}^n\alpha_k\left(\frac{\lambda_k}{\lambda_1}\right)^px^Tv_k\right]
$$
In order to have estimate $\lambda_1$ we can consider
\begin{align*}
\frac{x^TA^pw}{x^TA^{p-1}w} &= \lambda_1\frac{\left[\alpha_1x^Tv_1 + \sum_{k=2}^n\alpha_k\left(\frac{\lambda_k}{\lambda_1}\right)^px^Tv_k\right]}{\left[\alpha_1x^Tv_1 + \sum_{k=2}^n\alpha_k\left(\frac{\lambda_k}{\lambda_1}\right)^{p-1}x^Tv_k\right]}\\
& = \lambda_1\frac{\left[1 + \frac{1}{\alpha_1x^Tv_1}\sum_{k=2}^n\alpha_k\left(\frac{\lambda_k}{\lambda_1}\right)^px^Tv_k\right]}{\left[1 + \frac{1}{\alpha_1x^Tv_1}\sum_{k=2}^n\alpha_k\left(\frac{\lambda_k}{\lambda_1}\right)^{p-1}x^Tv_k\right]},
\end{align*}
this quantity will converge towards $\lambda_1$ (provided $x$ and $w$ match the requirements).

As for estimating $v_1$ we can use the fact that $A^pw\approx \lambda_1^p\alpha_1v_1$ when $p$ is large enough, which is parallel to $v_1$. In order to avoid overflows we can modify slightly the iteration and normalize the estimated vector at each step. 

Let us denote by $v_1^{(p)}$ the estimation of $v_1$ at iteration $p$. The algorithm to estimate $v_1$ is as follows
- initialize $v_1^{0}=\frac{w}{\|w\|}$
- $v_1^{(p)} = \frac{Av_1^{(p-1)}}{\|Av_1^{(p-1)}\|}$


Once we have estimated $\lambda_1$ and $v_1$, we can repeat the process for $\lambda_2$ and $v_2$ by chosing an initial vector $w_2 = w - \left<w, v_1\right>v_1$ which gives 
$$
Aw_2 = \sum_{k=2}^n\alpha_k\lambda_kv_k
$$


There are a number of issues with this method, the main ones being 
- the speed of convergence which can be reduced if $\lambda_2$ is close to $\lambda_1$ 
- numerical stability issues when having large or small eigenvalues.
- the eigenvalues/eigenvectors are computed sequentially

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## 3.2 QR method

### 3.2.1
Prove that the matrix $B_1=(I-v_1v_1^T)A$ has eigenvalues $0, \lambda_2, ..., \lambda_n$, with corresponding eigenvectors $v_1, (I-v_1v_1^T)v_2, ..., (I-v_1v_1^T)v_n$. How about the matrices $B_2=(I-v_2v_2^T)(I-v_1v_1^T)A, B_3=(I-v_3v_3^T)(I-v_2v_2^T)(I-v_1v_1^T)A$, etc. when supposing that $v_1, v_2...$ are orthogonal ?

Using question 1 (you can show the operator $I-v^Tv$ is indeed the basis operation of Gram-Schmidt), reformulate the following algorithm using the QR decomposition of $AV^{(k)}$ (denoted by $V^{(k+1)}R^{(k+1)}$):
- Start with $v_1^{(0)}, v_2^{(0)}, ..., v_n^{(0)}=V^{(0)}$
- Iterate for $k=0, 1, ... $
  - $v_1^{(k+1)}=Av_1^{(k)}$ and normalize $v_1^{(k+1)}$
  - $v_2^{(k+1)} = (I-v_1^{(k+1)}v_1^{(k+1)T})Av_1^{(k)}$ and normalize $v_2^{(k+1)}$
  - ...
  - $v_n^{(k+1)} = (I-v_{n-1}^{(k+1)}v_{n-1}^{(k+1)T})...(I-v_1^{(k+1)}v_1^{(k+1)T})Av_1^{(k)}$ and normalize $v_n^{(k+1)}$
  


_Type your answer here, replacing this text._

We have $(I-v_1v_1^T)Av_1 = \lambda_1 v_1 - \lambda_1 v_1 v_1^T v_1 = \lambda_1v_1 - \lambda_1v_1 = 0$, therefore $v_1$ is an eigenvector of $(I-v_1v_1^T)A$ associated with eigenvalue 0.

For $k>1$, we have
\begin{align*}
(I-v_1v_1^T)A(I-v_1v_1^T)v_k &= (I-v_1v_1^T)(A-Av_1v_1^T)v_k\\ &
= (A  - v_1v_1^TA - Av_1v_1^T + v_1v_1^TAv_1v_1^T)v_k\\ 
&= (A -v_1v_1^TA - \lambda_1v_1v_1^T+ \lambda_1v_1v_1^Tv_1v_1^T)v_k\\
&= (I-v_1v_1^T)Av_k \\
&= \lambda_k(I-v_1v_1^T)v_k
\end{align*}


Using a similar reasoning, matrix $B_2$ will have $v_2$ as eigenvector associated to eigenvalue 0, and since $v_1$ and $v_2$ are independant, the 0 eigenvalue will have a multiplicity of 2, its largest eigenvalue (in absolute value) becomes $\lambda_3$. For $k>2$, the eigenvectors associated to $\lambda_k$ will be $(I-v_2v_2^T)(I-v_1v_1^T)v_k$. With a similar reasoning $B_k$ will have a zero eigenvalue of multiplicity $k$ and its largest eigenvalue (in absolute value) will be $\lambda_k$.


The algorithm described is the same as question 1. Since we are dealing with unit vector, we can rewrite the Gram-Schmidt step as:
$$
v - P_uv = v -(u^Tv)u = v - u(u^T)v = (I-uu^T)v
$$
(since $u^Tv$ is a scalar, it is possible to move it around).

This transforms the basis $(Av_1^{(k)}, Av_2^{(k)}..., Av_n^{(k)})$ into an orthonormal basis $(v_1^{(k+1}, v_2^{(k+1}, ..., v_n^{(k+1})$. Therefore the algorithm described can be rewritten as
- Initialize $(v_1^{(0)}, v_2^{(0)}, ..., v_n^{(0)})=V^{(0)}$
- Iterate for $k=0, 1, ... $
  - Compute $AV^{(k)}$ and perform its QR decomposition $AV^{(k)}=V^{(k+1)}R^{(k+1)}$

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### 3.2.2

How does the algorithm of question 3.2.1 relates to the one from question 3.1 ?


_Type your answer here, replacing this text._

The method found at the previous question performs the power iteration simultaneously on all estimates of the eigenvectors (instead of one at a time), allowing a reduced number of computations (provided the method actually converges to the desired solution, which we will not prove here).

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### 3.2.3

Let us introduce the matrix $A^{(k)}=V^{(k)T}AV^{(k)}$, where $(v_1^{(k)}, v_2^{(k)}, ..., v_n^{(k)})=V^{(k)}$. Let us also introduce the QR decomposition of $A^{(k)}=Q^{(k+1)}R^{(k+1)}$.

Prove that $A^{(k+1)} = R^{(k+1)}Q^{(k+1)}$. What is the interest of this sequence of matrices $A^{(k)}$ regarding the eigenvalues of $A$ ?

Hint: remember the QR decomposition is unique.

_Type your answer here, replacing this text._

We have $AV^{(k)} = V^{(k+1)}R^{(k+1)}$, with $V^{(k)}$ and $V^{(k+1)}$ being orthogonal, therefore
$$
A^{(k)}=V^{(k)T}V^{(k+1)}R^{(k+1)},
$$
and 
$$
A^{(k+1)}=V^{(k+1)T}AV^{(k+1)} = V^{(k+1)T}AV^{(k)}V^{(k)T}V^{(k+1)} = V^{(k+1)T}V^{(k+1)}R^{(k+1)}V^{(k)T}V^{(k+1)} = R^{(k+1)}V^{(k)T}V^{(k+1)}.
$$

By unicity of the QR decomposition, and since $V^{(k)T}V^{(k+1)}$ is orthogonal (as product of orthogonal matrices) the first result becomes $A^{(k)}=Q^{(k+1)}R^{(k+1)}$, with $Q^{(k+1)}=V^{(k)T}V^{(k+1)}$.

In the second relationship this becomes $A^{(k+1)}=R^{(k+1)}Q^{(k+1)}$.

Matrices $A^{(k)}$ are similar to $A$ and have the same eigenvalues.

<!-- END QUESTION -->

### 3.2.4

Implement the algorithm that generates the sequence $A^{(k)}$. The diagonal elements of $A^{(k)}$ will converge towards the eigenvalues of $A$ (you do not need to prove this result).

In [ ]:
def eigendecomposition_qr(A, num_iter=20):
    """
    Computes the eigenvalues of A using the QR iteration method
    
    Parameters:
      - A: input nxn matrix
      - num_iter: number of QR iterations to perform
      
    Returns:
      - a vector containing the eigenvalues
    """
    Ak = A
    # BEGIN SOLUTION
    for k in range(num_iter):
        q,r = qr_decomp_hh(Ak)
        Ak = r@q
    return np.diag(Ak) #, r, q
    # END SOLUTION

In [ ]:
grader.check("q3p24")

Despite the assumption about disctinct eigenvalues, you can verify the eigenvalues of the matrix studied in week 7 are computed correctly:

In [ ]:
B = np.array([[1, 0.25, 0], [0, 0.5, 0], [0, 0.25, 1]])

In [ ]:
eigendecomposition_qr(B, 150)

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Upload your notebook and separate pdf for theoretical questions if needed

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)